In [1]:
import numpy
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
filename = "subtitle.txt"
raw_text = open(filename, 'r').read()

Using TensorFlow backend.


In [2]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 
raw_text = clean_text(raw_text)

In [3]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(raw_text.split('\n\n'))

In [4]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [5]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(GRU(128))
    model.add(Dropout(0.1))
    model.add(Dense(256, activation='relu'))
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 91, 10)            1740      
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               53376     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 174)               44718     
Total params: 132,858
Trainable params: 132,858
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(predictors, label, epochs=50, verbose=1)

Epoch 1/50
264/264 [==============================] - 1s 3ms/step - loss: 5.1601 - accuracy: 0.0114
Epoch 2/50
264/264 [==============================] - 1s 2ms/step - loss: 5.1514 - accuracy: 0.0379
Epoch 3/50
264/264 [==============================] - 0s 2ms/step - loss: 5.1397 - accuracy: 0.0417
Epoch 4/50
264/264 [==============================] - 0s 2ms/step - loss: 5.0918 - accuracy: 0.0417
Epoch 5/50
264/264 [==============================] - 0s 2ms/step - loss: 5.0239 - accuracy: 0.0417
Epoch 6/50
264/264 [==============================] - 0s 2ms/step - loss: 5.0146 - accuracy: 0.0417
Epoch 7/50
264/264 [==============================] - 0s 2ms/step - loss: 4.9725 - accuracy: 0.0417
Epoch 8/50
264/264 [==============================] - 0s 2ms/step - loss: 4.9240 - accuracy: 0.0417
Epoch 9/50
264/264 [==============================] - 0s 2ms/step - loss: 4.8959 - accuracy: 0.0417
Epoch 10/50
264/264 [==============================] - 0s 2ms/step - loss: 4.8829 - accuracy: 0.0417

In [7]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [11]:
print (generate_text("This is", 5, model, max_sequence_len))
print (generate_text("Potter", 5, model, max_sequence_len))
print (generate_text("can", 5, model, max_sequence_len))

This Is Scattered By The Way Pathetic
Potter Im Ron By The Way
Can You Cant Ron Pleasure You


In [12]:
!pip install gTTS
from gtts import gTTS
import os

mytext = generate_text("Potter", 3, model, max_sequence_len)
language = 'en'
  
myobj = gTTS(text=mytext, lang=language, slow=False)
  
myobj.save("predicted_audio.mp3")